In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np
from sklearn.pr

In [43]:
model = load_model('model.h5')

with open('ohencoder.pkl','rb') as file:
    ohencoder = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [45]:
input_data = {
    'Credit Score':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'Num of Products':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}
CreditScore	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	Exited

In [24]:
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder()

In [47]:
#ohgeo = ohencoder.transform([[input_data['Geography']]]).toarray()
#ohgeo = ohencoder.fit_transform(input_data[['Geography']])
#geo_encoded_df = pd.DataFrame(ohgeo,columns = ohgeo.get_features_names_out(['Geography']))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'get_features_names_out'

In [49]:
# Assuming input_data = {'Geography': 'India'}

ohgeo = ohencoder.transform(pd.DataFrame([[input_data['Geography']]], columns=['Geography'])).toarray()
geo_encoded_df = pd.DataFrame(ohgeo, columns = ohencoder.get_feature_names_out(['Geography']))

In [51]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [59]:
input_df = pd.DataFrame([input_data])
input_df

,Credit Score,Geography,Gender,Age,Tenure,Balance,Num of Products,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [61]:
input_df = pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_df

,Credit Score,Geography,Gender,Age,Tenure,Balance,Num of Products,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [71]:
print(input_df['Gender'].unique())

[1]


In [96]:
input_df.drop('Geography',axis=1,inplace=True)
input_df

,Credit Score,Gender,Age,Tenure,Balance,Num of Products,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [98]:
input_df

,Credit Score,Gender,Age,Tenure,Balance,Num of Products,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [105]:
input_df.rename(columns={
    'Credit Score':'CreditScore',
    'Num of Products':'NumOfProducts'
},inplace=True)

In [107]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [109]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


array([[0.03653676]], dtype=float32)

In [111]:
prediction_proba = prediction[0][0]
prediction_proba

0.03653676

In [113]:
if prediction_proba >0.5:
    print('The Customer is likely to churn')
else:
    print('The Customer is not likley to churn')

The Customer is not likley to churn
